# Visual Search - Web Application

<img src="https://github.com/retkowsky/images/blob/master/cohere4.jpg?raw=true" width=400>
This notebook demonstrates how to build and deploy an interactive web application for multimodal visual search using Gradio. 

It combines Azure AI services (embeddings, search) with an intuitive user interface that supports both text-based and image-based search across a fashion catalog.

The application provides an end-to-end solution for deploying semantic search capabilities as a shareable web interface, enabling users to discover fashion items using natural language descriptions or reference images without requiring technical knowledge.

The notebook demonstrates five core capabilities:
- Web UI Framework - Gradio-based responsive interface with theme customization
- Text-to-Image Search - Find fashion items using natural language descriptions
- Image-to-Image Search - Discover similar items by uploading a reference image
- Configurable Results - Adjustable number of results (1-20) via slider controls
- Results Gallery - Grid-based image display with similarity scores and download functionality

In [1]:
#%pip install gradio

In [25]:
import gradio as gr
import io
import json
import os
import requests
import sys
import tempfile
import time

from azure.ai.inference import EmbeddingsClient, ImageEmbeddingsClient
from azure.ai.inference.models import ImageEmbeddingInput
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.storage.blob import BlobServiceClient
from azure.search.documents.models import VectorizedQuery
from datetime import datetime
from dotenv import load_dotenv
from PIL import Image
from typing import List, Tuple

In [3]:
print(f"Python version: {sys.version}")

Python version: 3.10.18 (main, Jun  5 2025, 13:14:17) [GCC 11.2.0]


In [4]:
print(f"Today is {datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 11-Dec-2025 14:03:04


# Settings

In [5]:
load_dotenv("azure.env")

# Cohere v4
api_key = os.getenv("api_key")
endpoint = os.getenv("endpoint")

# Azure AI Search
azure_search_endpoint = os.getenv("azure_search_endpoint")
azure_search_key = os.getenv("azure_search_key")

# Azure storage account
blob_connection_string = os.getenv("blob_connection_string")
container_name = os.getenv("container_name")

deployment_name = "embed-v-4-0"  # name of Cohere Embed 4 model as deployed in Microsoft Foundry
index_name = "fashion-demo-2025"

In [6]:
text_client = EmbeddingsClient(endpoint=endpoint,
                               credential=AzureKeyCredential(api_key),
                               model=deployment_name)

In [7]:
image_client = ImageEmbeddingsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(api_key),
    model=deployment_name,
)

# Helper

In [8]:
def get_text_embeddings(text: str) -> list[float]:
    """
    Generate text embeddings using Cohere V4 embedding model.
    
    Converts input text into a dense vector representation (embedding) that
    captures semantic meaning, suitable for similarity comparisons, clustering,
    or retrieval tasks. Uses the configured deployment and text client for
    Azure OpenAI API access.
    
    Args:
        text (str): The input text to embed. Must not be empty or whitespace-only.
    
    Returns:
        list[float]: A list of floats representing the embedding vector for the input text.
    """
    if not text or not text.strip():
        raise ValueError("Input text cannot be empty")

    try:
        response = text_client.embed(
            input=[text.strip()],
            model=deployment_name,
        )
        return response.data[0].embedding
        
    except Exception as e:
        print("❌ Error")
        logging.error(f"Failed to generate embeddings: {e}")
        raise

In [9]:
def get_local_image_embeddings(image_path: str) -> list[float]:
    """
    Generate embeddings for a local image file using Cohere Embed 4.
    
    Converts an image file into a dense vector representation (embedding) that
    captures visual content and semantic meaning. The embedding can be used for
    image similarity comparisons, visual search, or multimodal retrieval tasks.
    
    Args:
        image_path (str): The file path to a local JPG image file to embed.
    
    Returns:
        list[float]: A list of floats representing the embedding vector for the input image.
    """
    try:
        image_input = ImageEmbeddingInput.load(
            image_file=image_path,
            image_format="jpg",
        )
        response = image_client.embed(input=[image_input])
        return response.data[0].embedding
        
    except Exception as e:
        print("❌ Error")
        logging.error(f"Failed to generate embeddings: {e}")
        raise

In [10]:
def get_image_embeddings_from_blob(blob_name: str) -> list[float]:
    """
    Generate embeddings with Cohere Embed 4 for an image stored in Azure Blob Storage.
    
    Downloads an image blob to a temporary local file, generates embeddings using
    Azure OpenAI's vision model, and cleans up the temporary file. This function
    bridges blob storage access with the embedding generation pipeline for images
    hosted in cloud storage.
    
    Args:
        blob_name (str): The name/path of the image blob in the configured container.
    
    Returns:
        list[float]: A list of floats representing the embedding vector for the blob image.
    """
    try:
        # Download blob to temporary file
        blob_service_client = BlobServiceClient.from_connection_string(
            blob_connection_string)
        blob_client = blob_service_client.get_blob_client(
            container=container_name, blob=blob_name)

        # Create temp file with proper extension
        with tempfile.NamedTemporaryFile(delete=False,
                                         suffix='.jpg') as tmp_file:
            tmp_path = tmp_file.name
            blob_data = blob_client.download_blob()
            blob_data.readinto(tmp_file)

        # Generate embeddings using your existing function
        embeddings = get_local_image_embeddings(tmp_path)

        # Clean up temp file
        os.unlink(tmp_path)

        return embeddings

    except Exception as e:
        print("❌ Error")
        logging.error(f"Failed to process blob image: {e}")
        raise

In [11]:
def get_index_stats(index_name: str) -> tuple[int, int]:
    """
    Retrieve statistics for an Azure AI Search index.
    
    Fetches index metadata including document count and storage size from the
    Azure AI Search service. Uses the REST API directly to query index statistics,
    providing insights into index usage and capacity.
    
    Args:
        index_name (str): The name of the search index to retrieve statistics for.
    
    Returns:
        tuple[int, int]: A tuple containing:
            - document_count (int): The total number of documents in the index
            - storage_size (int): The total storage size in bytes used by the index
    
    Raises:
        requests.exceptions.RequestException: If the API request fails
        KeyError: If expected fields are missing from the response
    """
    url = f"{azure_search_endpoint}/indexes/{index_name}/stats?api-version=2025-09-01"
    headers = {
        "Content-Type": "application/json",
        "api-key": azure_search_key,
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        stats = response.json()
        print(f"✅ Azure AI Search index statistics for: {index_name}\n")
        print(json.dumps(stats, indent=2))
        
        document_count = stats['documentCount']
        storage_size = stats['storageSize']
        
        return document_count, storage_size
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
        raise
    except KeyError as e:
        print(f"❌ Missing expected field in response: {e}")
        raise
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        raise

In [12]:
def prompt_search_gradio(prompt: str, topn: int = 5) -> List:
    """
    Prompt search for the gradio webapp
    """
    results_list = []
    images_list = []
    imgsize = 360

    # Initialize the Azure AI Search client
    search_client = SearchClient(azure_search_endpoint, index_name,
                                 AzureKeyCredential(azure_search_key))

    # Perform vector search
    query_vector = get_text_embeddings(prompt)

    request = search_client.search(None,
                                   vector_queries=[
                                       VectorizedQuery(
                                           vector=query_vector,
                                           k_nearest_neighbors=topn,
                                           fields="imagevector")
                                   ])

    results = [(doc["imagefile"], doc["@search.score"]) for doc in request]

    for result in results:
        image_file = result[0]
        results_list.append(image_file)

    for image_file in results_list:
        blob_client = container_client.get_blob_client(image_file)
        blob_image = blob_client.download_blob().readall()
        img = Image.open(io.BytesIO(blob_image)).resize((imgsize, imgsize))
        images_list.append(img)

    return images_list

In [13]:
def image_search_gradio(imagefile: str, topn: int = 5) -> List:
    """
     Performs an image-based search for the Gradio web app and returns the top N image results.

    Args:
        imagefile (str): The image file to be vectorized and used for the search.
        topn (int, optional): The number of top results to return. Defaults to 5.

    Returns:
        list: A list of PIL Image objects representing the top N search results.
    """
    results_list = []
    images_list = []
    imgsize = 360

    # Azure AI search client
    search_client = SearchClient(azure_search_endpoint, index_name,
                                 AzureKeyCredential(azure_search_key))

    query_vector = get_local_image_embedding(imagefile)
    request = search_client.search(None,
                                   vector_queries=[
                                       VectorizedQuery(
                                           vector=query_vector,
                                           k_nearest_neighbors=topn,
                                           fields="imagevector")
                                   ])

    # Assuming the search results include cosine similarity scores
    results = [(doc["imagefile"], doc["@search.score"]) for doc in request]

    print("\033[1;34m", end="")
    for idx, (filename, score) in enumerate(results, start=1):
        print(f"Top {idx:02}: {filename} with Cosine Similarity = {score:.5}")
        images_list.append(filename)

    for image_file in images_list:
        blob_client = container_client.get_blob_client(image_file)
        blob_image = blob_client.download_blob().readall()
        img = Image.open(io.BytesIO(blob_image)).resize((imgsize, imgsize))
        results_list.append(img)

    return results_list

In [14]:
def delete_index(index_name: str) -> None:
    """
    Deletes an Azure AI Search index.

    Args:
        index_name (str): The name of the index to be deleted.

    Returns:
        None
    """
    search_client = SearchIndexClient(
        endpoint=azure_search_endpoint,
        credential=AzureKeyCredential(azure_search_key))

    print(f"🗑️ Deleting the Azure AI Search index: {index_name}")
    search_client.delete_index(index_name)
    print("✅ Done")

# Blob storage

In [15]:
# Connect to Blob Storage
blob_service_client = BlobServiceClient.from_connection_string(
    blob_connection_string)
container_client = blob_service_client.get_container_client(container_name)
blobs = container_client.list_blobs()

first_blob = next(blobs)
blob_url = container_client.get_blob_client(first_blob).url
print(f"URL of the first blob: {blob_url}")

URL of the first blob: https://azurestorageaccountsr.blob.core.windows.net/fashionimages/fashion/0390469004.jpg


# Azure AI Search

In [16]:
try:
    # Setting the Azure AI Search client
    print("Setting the Azure AI Search client")
    search_client = SearchIndexClient(
        endpoint=azure_search_endpoint,
        credential=AzureKeyCredential(azure_search_key))
    print("✅ Done")
    print(search_client)

except:
    print(
        f"❌ Request failed. Cannot create Azure AI Search client: {azure_search_endpoint}"
    )

Setting the Azure AI Search client
✅ Done


In [17]:
document_count, storage_size = get_index_stats(index_name)

✅ Azure AI Search index statistics for: fashion-demo-2025

{
  "@odata.context": "https://azureaisearch-sr.search.windows.net/$metadata#Microsoft.Azure.Search.V2025_09_01.IndexStatistics",
  "documentCount": 2000,
  "storageSize": 31724997,
  "vectorIndexSize": 12402620
}


In [18]:
print(f"📊 Number of documents in the index = {document_count:,}")
print(f"💾 Size of the index = {storage_size / (1024 * 1024):.2f} MB")

📊 Number of documents in the index = 2,000
💾 Size of the index = 30.26 MB


# Webapp

In [19]:
def prompt_search_gradio(prompt: str, topn: int = 10) -> List:
    """
    Prompt search for the gradio webapp
    """
    results_list = []
    images_list = []
    imgsize = 360
    # Initialize the Azure AI Search client
    search_client = SearchClient(azure_search_endpoint, index_name,
                                 AzureKeyCredential(azure_search_key))

    # Perform vector search
    query_vector = get_text_embeddings(prompt)
    request = search_client.search(None,
                                   vector_queries=[
                                       VectorizedQuery(
                                           vector=query_vector,
                                           k_nearest_neighbors=topn,
                                           fields="imagevector")
                                   ])
    results = [(doc["imagefile"], doc["@search.score"]) for doc in request]

    # Build gallery with images and captions
    for image_file, score in results:
        blob_client = container_client.get_blob_client(image_file)
        blob_image = blob_client.download_blob().readall()
        img = Image.open(io.BytesIO(blob_image)).resize((imgsize, imgsize))
        images_list.append((img, f"{os.path.basename(image_file)}\nscore = {score:.4f}"))

    return images_list

In [20]:
def image_search_gradio(imagefile: str, topn: int = 10) -> List:
    """
     Performs an image-based search for the Gradio web app and returns the top N image results.
    Args:
        imagefile (str): The image file to be vectorized and used for the search.
        topn (int, optional): The number of top results to return. Defaults to 5.
    Returns:
        list: A list of tuples (PIL Image, caption) representing the top N search results.
    """
    results_list = []
    images_list = []
    imgsize = 360
    # Azure AI search client
    search_client = SearchClient(azure_search_endpoint, index_name,
                                 AzureKeyCredential(azure_search_key))
    query_vector = get_local_image_embeddings(imagefile)
    request = search_client.search(None,
                                   vector_queries=[
                                       VectorizedQuery(
                                           vector=query_vector,
                                           k_nearest_neighbors=topn,
                                           fields="imagevector")
                                   ])
    # Assuming the search results include cosine similarity scores
    results = [(doc["imagefile"], doc["@search.score"]) for doc in request]
    print("\033[1;34m", end="")
    for idx, (filename, score) in enumerate(results, start=1):
        print(f"Top {idx:02}: {filename} with Cosine Similarity = {score:.4}")

    # Build gallery with images and captions
    for image_file, score in results:
        blob_client = container_client.get_blob_client(image_file)
        blob_image = blob_client.download_blob().readall()
        img = Image.open(io.BytesIO(blob_image)).resize((imgsize, imgsize))
        results_list.append((img, f"{os.path.basename(image_file)}\nScore = {score:.4f}"))

    return results_list

In [21]:
def unified_search(text_query: str, 
                   image_input, 
                   search_type: str, 
                   top_n: int):
    """
    Unified search function that handles both text and image searches.
    Returns a tuple of (gallery_results, status_message)
    """
    try:
        if search_type == "Text":
            if not text_query or text_query.strip() == "":
                return [], "❌ Please enter a text query"
            start = time.time()
            results = prompt_search_gradio(text_query, top_n)
            elapsed = time.time() - start
            # Results should be a list of tuples (image, caption) for the gallery
            if isinstance(results, (list, tuple)):
                return results, f"✅ Text search completed in {elapsed:.2f} seconds"
            return results, f"✅ Text search completed in {elapsed:.2f} seconds"
        
        elif search_type == "Image":
            if image_input is None:
                return [], "❌ Please upload an image"
            start = time.time()
            results = image_search_gradio(image_input, top_n)
            elapsed = time.time() - start
            # Results should be a list of tuples (image, caption) for the gallery
            if isinstance(results, (list, tuple)):
                return results, f"✅ Image search completed in {elapsed:.2f} seconds"
            return results, f"✅ Image search completed in {elapsed:.2f} seconds"
        
        else:
            return [], "❌ Please select a search type"
    
    except Exception as e:
        return [], f"❌ Error during search: {str(e)}"

with gr.Blocks() as webapp:
    
    # Header
    gr.HTML("""
        <div id="header">
            <h1>🔍 AI-Powered Visual Search using Cohere Embed 4</h1>
            <p>Powered by Microsoft Foundry</p>
        </div>
    """)
    
    with gr.Row():
        with gr.Column():
            with gr.Group(elem_classes="search-container"):
                gr.Markdown("## 🎯 Search Setup")
                
                # Search Type Selection
                with gr.Group():
                    gr.Markdown("### 1️⃣ Select search type")
                    search_type = gr.Radio(
                        choices=["Text", "Image"],
                        value="Text",
                        label="Search by:",
                        interactive=True
                    )
                
                # Text Input Section
                with gr.Group(visible=True) as text_group:
                    gr.Markdown("### 2️⃣ Enter text description")
                    text_input = gr.Textbox(
                        label="What are you looking for?",
                        placeholder="e.g., a red dress, blue shirt...",
                        lines=1,
                        interactive=True
                    )
                
                # Image Upload Section
                with gr.Group(visible=False) as image_group:
                    gr.Markdown("### 2️⃣ Upload reference image")
                    image_input = gr.Image(
                        label="Upload Image",
                        type="filepath",
                        interactive=True
                    )
                    gr.Markdown("*Upload an image to find visually similar results*")
                
                # Number of Results
                with gr.Group():
                    gr.Markdown("### 3️⃣ Number of images")
                    top_n = gr.Slider(
                        minimum=1,
                        maximum=20,
                        value=8,
                        step=1,
                        label="Number of results to display"
                    )
                
                # Search Button
                search_btn = gr.Button(
                    "SEARCH IMAGES",
                    variant="primary",
                    size="lg",
                    elem_id="search-button",
                    scale=2
                )
                
                # Status Message
                status_msg = gr.Markdown(
                    value="",
                    elem_classes="status-message"
                )
    
    # Results Section
    with gr.Row():
        with gr.Column():
            with gr.Group(elem_classes="gallery-container"):
                gr.Markdown("## 🎯 Visual search results")
                results_gallery = gr.Gallery(
                    label="",
                    show_label=False,
                    columns=4,
                    rows=5,
                    object_fit="cover"
                )
    
    # Footer
    gr.HTML("""
        <footer>
            <p>Powered by Microsoft Foundry | Built with Gradio</p>
        </footer>
    """)
    
    # Toggle visibility based on search type
    def toggle_search_input(search_choice):
        return gr.Group(visible=(search_choice == "Text")), gr.Group(visible=(search_choice == "Image"))
    
    search_type.change(
        fn=toggle_search_input,
        inputs=search_type,
        outputs=[text_group, image_group]
    )
    
    # Event handler for unified search
    search_btn.click(
        fn=unified_search,
        inputs=[text_input, image_input, search_type, top_n],
        outputs=[results_gallery, status_msg]
    )



In [22]:
webapp.launch(share=True, show_error=True, theme=gr.themes.Soft())

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://e343a345797547c93a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Post processing

We can delete the index if needed

In [23]:
get_index_stats(index_name)

✅ Azure AI Search index statistics for: fashion-demo-2025

{
  "@odata.context": "https://azureaisearch-sr.search.windows.net/$metadata#Microsoft.Azure.Search.V2025_09_01.IndexStatistics",
  "documentCount": 2000,
  "storageSize": 31724997,
  "vectorIndexSize": 12402620
}


(2000, 31724997)

In [24]:
#delete_index(index_name)